<a href="https://colab.research.google.com/github/Kovasluksas/Kursinis-darbas/blob/main/kursinis(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import json
from abc import ABC, abstractmethod

# Excepciones Personalizadas
class StackError(Exception):
    pass

class StackUnderflowError(StackError):
    """Error cuando se hace pop en un stack vacío"""
    pass

class StackOverflowError(StackError):
    """Error cuando se hace push en un stack lleno"""
    pass


# Interfaz y Stack Base
class IStack(ABC):
    """
    A stack interface with the basic stack operations.

    Methods:
    ----------
    push(item) -- Adds an item to the stack.
    pop() -- Removes and returns the top item from the stack.
    peek() -- Returns the top item without removing it.
    is_empty() -- Checks if the stack is empty.
    """
    @abstractmethod
    def push(self, item):
        pass

    @abstractmethod
    def pop(self):
        pass

    @abstractmethod
    def peek(self):
        pass

    @abstractmethod
    def is_empty(self):
        pass


class SimpleStack(IStack):
  def __init__(self, max_size=None):

        self._items = []
        self.max_size = max_size

  def push(self, item):

        if self.is_full():
            raise StackOverflowError("Stack is full")
        self._items.append(item)

  def pop(self):

        if self.is_empty():
            raise StackUnderflowError("Cannot pop from empty stack")
        return self._items.pop()

  def peek(self):

        return self._items[-1] if not self.is_empty() else None

  def is_empty(self):

        return len(self._items) == 0

  def is_full(self):

        return self.max_size is not None and len(self._items) >= self.max_size


# Decoradores
class StackDecorator(IStack):

    def __init__(self, stack: IStack):
        self._stack = stack

    def push(self, item):
        self._stack.push(item)

    def pop(self):
        return self._stack.pop()

    def peek(self):
        return self._stack.peek()

    def is_empty(self):
        return self._stack.is_empty()


class LoggingDecorator(StackDecorator):

    def push(self, item):
        print(f"[LOG] Push: {item}")
        super().push(item)

    def pop(self):
        item = super().pop()
        print(f"[LOG] Pop: {item}")
        return item


class PersistentDecorator(StackDecorator):

    def __init__(self, stack: IStack, filename: str):
        super().__init__(stack)
        self.filename = filename

    def push(self, item):
        super().push(item)
        self._save_to_file()

    def pop(self):
        item = super().pop()
        self._save_to_file()
        return item

    def _save_to_file(self):
        with open(self.filename, 'w') as f:
            json.dump(self._stack._items, f)


# Función Principal para Demo
def main():
    stack = LoggingDecorator(
        PersistentDecorator(
            SimpleStack(max_size=5),
            "stack_data.json"
        )
    )

    # Operaciones de Demo
    stack.push(10)
    stack.push(20)
    stack.pop()
    stack.push(30)
    print(f"Estado final: {stack.peek()}")


# Ejecutar Demo
if __name__ == "__main__":
    print("\n=== Running Demo ===")
    main()

    #mandarlo a un file externo
    #butiniai que el user pueda poner el numero de push, pop, etc. QUE PUEDA ELEGIR
    #TESTU IRGI NEPAMIRSK



=== Running Demo ===
[LOG] Push: 10
[LOG] Push: 20
[LOG] Pop: 20
[LOG] Push: 30
Estado final: 30
